In [13]:
import os
import torch
import pyreadstat
import numpy as np
import pandas as pd
import networkx as nx
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch_geometric.transforms import RandomNodeSplit

In [14]:
def get_datasets():
    # Preparing the Graph Dataset
    DATASET_DIR = '/home/shussein/NetCO/GNNs/data/COPD/SparsifiedNetworks'
    dataset_name = 'trimmed_fev1_0.515_0.111_adj.csv'

    graph_adj_file = os.path.join(DATASET_DIR, dataset_name)
    graph_adj = pd.read_csv(graph_adj_file, index_col=0).to_numpy()
    nodes_names = pd.read_csv(graph_adj_file, index_col=0).index.tolist()

    original_dataset_sid = pd.read_csv('/home/shussein/NetCO/GNNs/data/COPD/SparsifiedNetworks/fev1_X.csv', index_col=0)
    original_dataset_sid.index.name = 'sid'

    clinical_variables, meta = pyreadstat.read_sas7bdat("/home/shussein/NetCO/Data/COPDGene_P1P2P3_SM_NS_Long_Oct22.sas7bdat")
    clinical_variables = clinical_variables.set_index('sid')
    # Filtering on Visit Number
    clinical_variables = clinical_variables[clinical_variables['visitnum'] == 2.0]

    clinical_variables_comorbidities = ['Angina', 'CongestHeartFail', 'CoronaryArtery', 'HeartAttack', 'PeriphVascular',
                                        'Stroke', 'TIA', 'Diabetes',
                                        'Osteoporosis', 'HighBloodPres', 'HighCholest', 'CognitiveDisorder',
                                        'MacularDegen', 'KidneyDisease', 'LiverDisease']
    clinical_variables = clinical_variables.assign(comorbidities=clinical_variables[clinical_variables_comorbidities].sum(axis=1))

    complete_original_dataset = pd.merge(clinical_variables, original_dataset_sid, left_index=True, right_index=True)
    clinical_variables = complete_original_dataset[clinical_variables.columns]

    complete_original_dataset['finalgold_visit'].fillna(0, inplace=True)

    complete_original_dataset.drop(complete_original_dataset[complete_original_dataset['finalgold_visit'] == -1].index, inplace=True)
    original_dataset_sid = original_dataset_sid[original_dataset_sid.index.isin(complete_original_dataset.index.tolist())]
    clinical_variables_cols = ['gender', 'age_visit', 'Chronic_Bronchitis', 'PRM_pct_emphysema_Thirona',
                               'PRM_pct_normal_Thirona', 'Pi10_Thirona', 'comorbidities']

    # Unifying Classes 0, -1, {1, 2} -> 1, {3, 4} -> 2
    complete_original_dataset['finalgold_visit'] = np.where(complete_original_dataset['finalgold_visit'] == 2, 1,
                                                            complete_original_dataset['finalgold_visit'])
    complete_original_dataset['finalgold_visit'] = np.where((complete_original_dataset['finalgold_visit'] == 3) |
                                                            (complete_original_dataset['finalgold_visit'] == 4), 2,
                                                            complete_original_dataset['finalgold_visit'])

    original_dataset_sid['finalgold_visit'] = complete_original_dataset['finalgold_visit']

    graph = nx.from_numpy_array(graph_adj)

    nodes_features = []
    for node_name in nodes_names:
        node_features = []
        for clinical_variable in clinical_variables_cols:
            node_features.append(
                abs(original_dataset_sid[node_name].corr(
                    complete_original_dataset[clinical_variable].astype('float64'))))
        nodes_features.append(node_features)

    features = np.array(nodes_features)
    graph.remove_edges_from(nx.selfloop_edges(graph))

    x = np.zeros(features.shape)
    graph_nodes = list(graph.nodes)
    for m in range(features.shape[0]):
        x[graph_nodes[m]] = features[m]
    x = torch.from_numpy(x).float()

    # Edges Indexes
    edge_index = np.array(list(graph.edges))
    edge_index = np.concatenate((edge_index, edge_index[:, ::-1]), axis=0)
    edge_index = torch.from_numpy(edge_index).long().permute(1, 0)

    # Edges Weights
    edge_weight = np.array(list(nx.get_edge_attributes(graph, 'weight').values()))
    edge_weight = np.concatenate((edge_weight, edge_weight), axis=0)
    edge_weight = torch.from_numpy(edge_weight).float()

    dataset = Data(x=x, edge_index=edge_index, edge_attr=edge_weight, y=torch.zeros(len(nodes_names)))

    transform = RandomNodeSplit(num_val=5, num_test=5)
    data = transform(dataset)
    return data, original_dataset_sid

In [17]:
class GCN(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_num=2, dropout=True, **kwargs):
        super(GCN, self).__init__()
        self.layer_num = layer_num
        self.dropout = dropout
        self.conv_first = GCNConv(input_dim, hidden_dim)
        self.conv_hidden = nn.ModuleList([GCNConv(hidden_dim, hidden_dim) for i in range(layer_num - 2)])

    def forward(self, data):
        x, edge_index, edge_weight = data.x, data.edge_index, data.edge_attr
        x = self.conv_first(x, edge_index, edge_weight)
        x = F.relu(x)
        
        if self.dropout:
            x = F.dropout(x, training=self.training)

        for i in range(self.layer_num-2):
            x = self.conv_hidden[i](x.clone(), edge_index, edge_weight)
            x = F.relu(x)
            
            if self.dropout:
                x = F.dropout(x, training=self.training)
        return x

class Autoencoder(nn.Module): # TODO: Need to revise this architecture
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 8),  
            nn.ReLU(),
            nn.Linear(8, 4),         
            nn.ReLU(),
            nn.Linear(4, encoding_dim) 
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 4),
            nn.ReLU(),
            nn.Linear(4, 8),           
            nn.ReLU(),
            nn.Linear(8, input_dim)    
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

class DownstreamTaskNN(nn.Module):
    def __init__(self, input_size, hidden_dim1, hidden_dim2, output_dim):
        super(DownstreamTaskNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_dim1)
        self.bn1 = nn.BatchNorm1d(hidden_dim1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.bn2 = nn.BatchNorm1d(hidden_dim2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_dim2, output_dim)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.softmax(x)
        return x

class NeuralNetwork(nn.Module):
    def __init__(self, 
                 gcn_input_dim, 
                 gcn_hidden_dim, 
                 gcn_layer_num, 
                 ae_encoding_dim, 
                 nn_input_dim, 
                 nn_hidden_dim1, 
                 nn_hidden_dim2, 
                 nn_output_dim):
        super(NeuralNetwork, self).__init__()
        self.gcn = GCN(input_dim=gcn_input_dim, hidden_dim=gcn_hidden_dim, layer_num=gcn_layer_num)
        self.autoencoder = Autoencoder(input_dim=gcn_hidden_dim, encoding_dim=ae_encoding_dim)
        self.predictor = DownstreamTaskNN(nn_input_dim, nn_hidden_dim1, nn_hidden_dim2, nn_output_dim)

    def forward(self, graph_data, original_dataset):
        nodes_embeddings = self.gcn(graph_data) 
        print("Dimenions of Embeddings after GCN")
        print(nodes_embeddings.shape)
        print(nodes_embeddings)
        nodes_embeddings_ae = self.autoencoder(nodes_embeddings)
        print("Dimensions of Embeddings after Autoencoder")
        print(nodes_embeddings_ae.shape)
        print(nodes_embeddings_ae)
        
        # Multiplying the original_dataset with the generated embeddings
        original_dataset_with_embeddings = original_dataset * nodes_embeddings_ae
        print("Dimensions of the Dataset after Integration")
        print(original_dataset_with_embeddings.shape)
        print(original_dataset_with_embeddings)
        predictions = self.predictor(original_dataset_with_embeddings)
        print("Dimensions of the Dataset after Prediction")
        print(predictions.shape)
        print(predictions)
        return predictions

graph_data, original_dataset = get_datasets()
# TODO: Need to split the 'original_dataset' into training and testing 
X = original_dataset.drop(['finalgold_visit'], axis=1)
y = original_dataset['finalgold_visit']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert to PyTorch Tensors
X_train_tensor = torch.FloatTensor(X_train_scaled)
y_train_tensor = torch.LongTensor(y_train.to_numpy())
X_test_tensor = torch.FloatTensor(X_test_scaled)
y_test_tensor = torch.LongTensor(y_test.to_numpy())

gcn_input_dim = graph_data.x.shape[1]
gcn_hidden_dim = 128
gcn_layer_num = 2
ae_encoding_dim = 1
nn_input_dim = X_train_tensor.shape[1]
nn_hidden_dim1 = 64
nn_hidden_dim2 = 16
nn_output_dim = 3 # Number of Classes

model = NeuralNetwork(gcn_input_dim=gcn_input_dim, gcn_hidden_dim=gcn_hidden_dim, gcn_layer_num=gcn_layer_num,
                      ae_encoding_dim=ae_encoding_dim,
                      nn_input_dim=nn_input_dim, nn_hidden_dim1=nn_hidden_dim1, nn_hidden_dim2=nn_hidden_dim2, nn_output_dim=nn_output_dim)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.030254, weight_decay=0.000100)

epochs_num = 16
# Training loop
for epoch in range(epochs_num):
    # Forward pass
    output = model(graph_data, X_train_scaled)
    # Compute loss
    loss = criterion(output, y_train_tensor)

    # Backward pass
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print loss for monitoring
    print(f"Epoch [{epoch+1}/{epochs_num}], Loss: {loss.item()}")
    
    print("*****************GRADIENTS********************")
    print("Gradients for the Model!!!! ")
    # Track gradients (gnn_model) and model parameters in TensorBoard
    for name, param in model.named_parameters():
        print(f'{name}.grad', param.grad, epoch)

Dimenions of Embeddings after GCN
torch.Size([27, 128])
tensor([[0.0654, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0782, 0.0000, 0.0000,  ..., 0.0591, 0.0000, 0.0000],
        [0.0372, 0.0000, 0.0000,  ..., 0.0000, 0.0353, 0.0000],
        ...,
        [0.0880, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.2048, 0.0000, 0.0000],
        [0.1143, 0.0000, 0.0000,  ..., 0.0570, 0.0000, 0.0000]],
       grad_fn=<MulBackward0>)
Dimensions of Embeddings after Autoencoder
torch.Size([27, 128])
tensor([[ 0.0784,  0.0464,  0.3894,  ..., -0.5173, -0.1933, -0.2545],
        [ 0.0786,  0.0465,  0.3891,  ..., -0.5171, -0.1935, -0.2545],
        [ 0.0786,  0.0465,  0.3892,  ..., -0.5172, -0.1935, -0.2545],
        ...,
        [ 0.0786,  0.0465,  0.3891,  ..., -0.5171, -0.1935, -0.2545],
        [ 0.0793,  0.0469,  0.3882,  ..., -0.5164, -0.1941, -0.2545],
        [ 0.0784,  0.0464,  0.3894,  ..., -0.5173, -0.1933, -0.2545]],
       grad_fn=<

TypeError: unsupported operand type(s) for *: 'numpy.ndarray' and 'Tensor'